In [1]:
!pip install reportlab

# ***FINAL REPORT GENERATOR***

In [1]:
!pip install --no-cache-dir \
torch==2.2.2+cu118 \
torchvision==0.17.2+cu118 \
torchaudio==2.2.2+cu118 \
--index-url https://download.pytorch.org/whl/cu118

!pip install timm==0.9.12 reportlab opencv-python-headless tqdm


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.1/819.1 MB 248.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 322.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 313.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 236.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 391.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 314.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 283.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 137.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 169.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 298.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 280.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204

In [1]:
!pip install --no-cache-dir numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 291.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
op

In [2]:
# ===============================================
# 🧠 Compact Brain Tumor Diagnostic Report Generator
# ===============================================

import os, torch, torch.nn as nn, torchvision.transforms as T
from PIL import Image
import numpy as np
import tensorflow as tf
import timm, cv2
from tqdm import tqdm
from collections import Counter
from google.colab import drive
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib.pagesizes import A4
width, height = A4


# ===============================================
# 1️⃣ Mount Drive
# ===============================================
!fusermount -u /content/drive 2>/dev/null || echo "Drive not mounted yet"
drive.mount('/content/drive', force_remount=True)

# ===============================================
# 2️⃣ Paths & Setup
# ===============================================
CLASS_NAMES = ['glioma', 'meningioma', 'notumor', 'pituitary']

base_dir = "/content/drive/MyDrive/final/working"
model_path = f"{base_dir}/swin_trained_checkpoint.pth"
gradcam_model_path = f"{base_dir}/fine_tuned_enhanced_model.h5"
unet_path = f"{base_dir}/attn_unet_brisc2025_t4_acc.pth"
image_folder = f"{base_dir}/test"
report_dir = f"{base_dir}/Brain_Tumor_Reports"

os.makedirs(report_dir, exist_ok=True)
gradcam_dir = os.path.join(report_dir, "gradcam_results")
overlay_dir = os.path.join(report_dir, "output_overlay")
os.makedirs(gradcam_dir, exist_ok=True)
os.makedirs(overlay_dir, exist_ok=True)

# Dynamic PDF naming (auto-increment)
base_pdf = os.path.join(report_dir, "Brain_Tumor_Report.pdf")
pdf_path = base_pdf
i = 1
while os.path.exists(pdf_path):
    pdf_path = os.path.join(report_dir, f"Brain_Tumor_Report({i}).pdf")
    i += 1

logo_path = f"{base_dir}/logo.jpg"
doclogo_path = f"{base_dir}/doclogo.jpg"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===============================================
# 3️⃣ Load Models
# ===============================================
try:
    model = timm.create_model('swin_base_patch4_window12_384', pretrained=False, num_classes=len(CLASS_NAMES))
except Exception:
    from torchvision.models import resnet18
    model = resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, len(CLASS_NAMES))

checkpoint = torch.load(model_path, map_location='cpu')
state_dict = checkpoint.get('model_state_dict') or checkpoint.get('model_state') or checkpoint
model.load_state_dict(state_dict, strict=False)
model.to(device).eval()

gradcam_model = tf.keras.models.load_model(gradcam_model_path, compile=False)
gradcam_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

class AttentionBlock(nn.Module):
    def __init__(self,F_g,F_l,F_int):
        super().__init__()
        self.W_g = nn.Sequential(nn.Conv2d(F_g,F_int,1),nn.BatchNorm2d(F_int))
        self.W_x = nn.Sequential(nn.Conv2d(F_l,F_int,1),nn.BatchNorm2d(F_int))
        self.psi = nn.Sequential(nn.Conv2d(F_int,1,1),nn.BatchNorm2d(1),nn.Sigmoid())
        self.relu = nn.ReLU(inplace=True)
    def forward(self,g,x):
        psi=self.relu(self.W_g(g)+self.W_x(x))
        psi=self.psi(psi)
        return x*psi

class AttentionUNet(nn.Module):
    def __init__(self,img_ch=3,out_ch=1):
        super().__init__()
        def conv(in_c,out_c):
            return nn.Sequential(
                nn.Conv2d(in_c,out_c,3,padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True),
                nn.Conv2d(out_c,out_c,3,padding=1), nn.BatchNorm2d(out_c), nn.ReLU(inplace=True))
        self.Maxpool=nn.MaxPool2d(2)
        self.Conv1, self.Conv2 = conv(3,64), conv(64,128)
        self.Conv3, self.Conv4, self.Conv5 = conv(128,256), conv(256,512), conv(512,1024)
        self.Up5, self.Up4, self.Up3, self.Up2 = (
            nn.ConvTranspose2d(1024,512,2,2), nn.ConvTranspose2d(512,256,2,2),
            nn.ConvTranspose2d(256,128,2,2), nn.ConvTranspose2d(128,64,2,2))
        self.Att5, self.Att4, self.Att3, self.Att2 = (
            AttentionBlock(512,512,256), AttentionBlock(256,256,128),
            AttentionBlock(128,128,64), AttentionBlock(64,64,32))
        self.Up_conv5, self.Up_conv4, self.Up_conv3, self.Up_conv2 = (
            conv(1024,512), conv(512,256), conv(256,128), conv(128,64))
        self.Conv_1x1=nn.Conv2d(64,out_ch,1)
    def forward(self,x):
        x1=self.Conv1(x);x2=self.Conv2(self.Maxpool(x1));x3=self.Conv3(self.Maxpool(x2))
        x4=self.Conv4(self.Maxpool(x3));x5=self.Conv5(self.Maxpool(x4))
        d5=self.Up5(x5);x4=self.Att5(d5,x4);d5=self.Up_conv5(torch.cat([x4,d5],1))
        d4=self.Up4(d5);x3=self.Att4(d4,x3);d4=self.Up_conv4(torch.cat([x3,d4],1))
        d3=self.Up3(d4);x2=self.Att3(d3,x2);d3=self.Up_conv3(torch.cat([x2,d3],1))
        d2=self.Up2(d3);x1=self.Att2(d2,x1);d2=self.Up_conv2(torch.cat([x1,d2],1))
        return torch.sigmoid(self.Conv_1x1(d2))

unet = AttentionUNet().to(device)
unet.load_state_dict(torch.load(unet_path, map_location=device))
unet.eval()

# ===============================================
# 4️⃣ Helper Functions
# ===============================================
def preprocess_pytorch(img_path):
    img = Image.open(img_path).convert('RGB')
    transform = T.Compose([
        T.Resize(int(384*1.14)),
        T.CenterCrop(384),
        T.ToTensor(),
        T.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
    return transform(img).unsqueeze(0)

def preprocess_tf(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(224,224))
    return np.expand_dims(img/255.0,axis=0), img

def grad_cam(img_array, model, layer_name='conv5_block16_concat'):
    grad_model = tf.keras.models.Model(inputs=model.inputs,
                                       outputs=[model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs, pooled_grads = conv_outputs[0].numpy(), pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        conv_outputs[:,:,i] *= pooled_grads[i]
    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= (heatmap.max() + 1e-8)
    return cv2.resize(heatmap, (224,224))

def overlay_heatmap(original_img, heatmap):
    if original_img.max() <= 1.0:
        original_img = (original_img * 255).astype(np.uint8)
    heatmap_color = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    return cv2.addWeighted(original_img, 0.6, heatmap_color, 0.4, 0)

# ===============================================
# 5️⃣ Predict + Generate Combined Visuals
# ===============================================
predictions, confidences = [], []
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg','.png','.jpeg'))]

for img_name in tqdm(image_files, desc="🧠 Processing Images"):
    img_path = os.path.join(image_folder, img_name)

    # Prediction
    input_tensor = preprocess_pytorch(img_path).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probs = torch.nn.functional.softmax(outputs, dim=1)[0]
        pred_idx = torch.argmax(probs).item()
        confidence = probs[pred_idx].item()
    predictions.append(CLASS_NAMES[pred_idx]); confidences.append(confidence)

    # Grad-CAM
    tf_array, disp_img = preprocess_tf(img_path)
    heatmap = grad_cam(tf_array, gradcam_model)
    grad_overlay = overlay_heatmap(disp_img, heatmap)
    cv2.imwrite(os.path.join(gradcam_dir, f"{os.path.splitext(img_name)[0]}_gradcam.jpg"),
                cv2.cvtColor(grad_overlay, cv2.COLOR_RGB2BGR))

    # U-Net segmentation
    img_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    inp_unet = torch.tensor(np.transpose(cv2.resize(img_rgb,(256,256))/255.0,(2,0,1))).unsqueeze(0).float().to(device)
    with torch.no_grad():
        mask = (unet(inp_unet) > 0.5).float().cpu().squeeze().numpy()
    mask_resized = cv2.resize(mask, (img_rgb.shape[1], img_rgb.shape[0]))
    yellow_mask = np.zeros_like(img_rgb); yellow_mask[...,0]=255; yellow_mask[...,1]=255
    overlay = np.where(mask_resized[...,None]>0.5,0.5*img_rgb+0.5*yellow_mask,img_rgb).astype(np.uint8)
    cv2.imwrite(os.path.join(overlay_dir, f"overlay_{img_name}"), cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

# ===============================================
# ===============================================
# 6️⃣ Generate Compact Auto-Paged PDF Report
# ===============================================
from reportlab.graphics.barcode import qr
from reportlab.graphics.shapes import Drawing
from reportlab.graphics import renderPDF

# --- Patient Info Inputs ---
patient_name = input("Enter Patient Name: ")
patient_age = input("Enter Patient Age: ")
patient_gender = input("Enter Patient Gender (M/F/Other): ")

# Optional: link to cloud report
#cloud_report_url = "https://drive.google.com/your-report-folder-link"

print("\n🧾 Generating compact report...")

# --- Dynamically name the report based on patient name ---
safe_name = patient_name.strip().replace(" ", "_")  # ensures no spaces
pdf_path = os.path.join(report_dir, f"{safe_name}_Brain_MRI_Report.pdf")

c = canvas.Canvas(pdf_path, pagesize=A4)


# --- Header ---
if os.path.exists(logo_path):
    c.drawImage(ImageReader(logo_path), 40, height - 120, width=130, height=100)
c.setFont("Helvetica-Bold", 26)
c.drawString(200, height - 90, "Brain Oncology Center")
c.setFont("Helvetica", 12)
c.drawString(200, height - 110, "Brain Tumor Diagnostic Report")
c.line(40, height - 130, width - 40, height - 130)



# --- Divider lines (unchanged) ---
c.line(40, height - 130, width - 40, height - 130)
c.line(40, height - 170, width - 40, height - 170)

# --- Patient Information (moved 0.2 cm up) ---
c.setFont("Helvetica", 13)
patient_info_y = height - 150 + 3 + 2.8   # +0.2 cm upward
c.drawString(
    50,
    patient_info_y,
    f"Patient Information :-  Name: {patient_name} | Age: {patient_age} | Gender: {patient_gender} "
)

# --- Column Titles ---
x_positions = [50, 200, 350]
img_size = 120
c.setFont("Helvetica", 12)
c.drawString(x_positions[0] + 10, height - 160, "MRI Image")
c.drawString(x_positions[1] + 10, height - 160, "Segmentation Overlay")
c.drawString(x_positions[2] + 10, height - 160, "Grad-CAM Heatmap")
c.line(40, height - 170, width - 40, height - 170)

# --- Draw Images Dynamically ---
y = height - 300
row_count = 0
for i, img_name in enumerate(image_files):
    orig = os.path.join(image_folder, img_name)
    seg = os.path.join(overlay_dir, f"overlay_{img_name}")
    grad = os.path.join(gradcam_dir, f"{os.path.splitext(img_name)[0]}_gradcam.jpg")

    if not all(os.path.exists(p) for p in [orig, seg, grad]):
        continue

    for j, p in enumerate([orig, seg, grad]):
        c.drawImage(ImageReader(p), x_positions[j], y, width=img_size, height=img_size)

    y -= 160
    row_count += 1

    # --- Page Break Handling ---
    if y < 100 and i != len(image_files) - 1:
        c.showPage()

        # --- Redraw layout for next page ---
        if os.path.exists(logo_path):
            c.drawImage(ImageReader(logo_path), 40, height - 120, width=130, height=100)
        c.setFont("Helvetica-Bold", 26)
        c.drawString(200, height - 90, "Brain Oncology Center")
        c.setFont("Helvetica", 12)
        c.drawString(200, height - 110, "Brain Tumor Diagnostic Report")
        c.line(40, height - 130, width - 40, height - 130)

        #qr_code = qr.QrCodeWidget(cloud_report_url)
        #bounds = qr_code.getBounds()
        #size = 70
        #w, h = bounds[2] - bounds[0], bounds[3] - bounds[1]
        #d = Drawing(size, size, transform=[size / w, 0, 0, size / h, 0, 0])
        #d.add(qr_code)
        #renderPDF.draw(d, c, width - size - 30, height - size - 40)

        #c.line(40, height - 130, width - 40, height - 130)
        #c.line(40, height - 170, width - 40, height - 170)

        c.setFont("Helvetica", 13)
        c.drawString(
            50,
            patient_info_y,
            f"Patient Information :-  Name: {patient_name} | Age: {patient_age} | Gender: {patient_gender} "
        )

        c.setFont("Helvetica", 12)
        c.drawString(x_positions[0] + 10, height - 160, "MRI Image")
        c.drawString(x_positions[1] + 10, height - 160, "Segmentation Overlay")
        c.drawString(x_positions[2] + 10, height - 160, "Grad-CAM Heatmap")
        c.line(40, height - 170, width - 40, height - 170)

        y = height - 300

# --- Summary Section ---
most_common_class = Counter(predictions).most_common(1)[0][0] if predictions else "Unknown"
avg_conf = np.mean([conf for p, conf in zip(predictions, confidences)
                    if p == most_common_class]) * 100 if predictions else 0

summary_y = 140
c.setFont("Helvetica-Bold", 16)
c.drawString(50, summary_y + 60, "Final Diagnostic Summary")
c.setFont("Helvetica", 12)
c.drawString(70, summary_y + 40, f"Predicted Tumor Type: {most_common_class.upper()}")
c.drawString(70, summary_y + 20, f"Average Confidence: {avg_conf:.2f}%")
c.drawString(70, summary_y, f"Images Processed: {len(image_files)}")

# --- Clinical Insight ---
c.setFont("Helvetica-Bold", 14)
c.drawString(50, summary_y - 40, "Clinical Insight")
c.setFont("Helvetica", 11)

if most_common_class.lower() == "glioma":
    text = [
        "• Origin: Glial cells of the brain or spinal cord.",
        "• Possible Causes: IDH, TP53, or EGFR mutations; radiation exposure.",
        "• Clinical Notes: Managed via surgery, radiotherapy, and chemotherapy.",
        "• Prognosis: Varies by grade; high-grade gliomas are aggressive."
    ]
elif most_common_class.lower() == "meningioma":
    text = [
        "• Origin: Meninges (protective membranes around the brain).",
        "• Possible Causes: NF2 gene loss, hormonal, or radiation factors.",
        "• Clinical Notes: Usually benign; surgery often curative.",
        "• Prognosis: Excellent if completely resected."
    ]
elif most_common_class.lower() == "pituitary":
    text = [
        "• Origin: Pituitary gland beneath the brain.",
        "• Possible Causes: Sporadic or MEN1 genetic syndrome.",
        "• Clinical Notes: May alter hormone levels; treated via surgery or medication.",
        "• Prognosis: Typically benign and treatable."
    ]
elif most_common_class.lower() == "notumor":
    text = [
        "• Interpretation: MRI analysis shows no evidence of tumor or mass lesion.",
        "• Clinical Notes: No focal abnormality detected; follow-up only if symptoms persist.",
        "• Prognosis: Normal intracranial findings; no oncologic concern at present."
    ]
else:
    text = ["• Unable to determine; insufficient classification confidence."]

y_pos = summary_y - 60
for line in text:
    c.drawString(70, y_pos, line)
    y_pos -= 15

# --- Doctor Logo ---
if os.path.exists(doclogo_path):
    logo_width, logo_height = 110, 80
    c.drawImage(ImageReader(doclogo_path), width - logo_width - 40, 40,
                width=logo_width, height=logo_height)

c.save()
print(f"✅ Report saved: {pdf_path}")


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
🧠 Processing Images:   0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)
🧠 Processing Images: 100%|██████████| 3/3 [00:09<00:00,  3.09s/it]


Enter Patient Name: Malik
Enter Patient Age: 56
Enter Patient Gender (M/F/Other): M

🧾 Generating compact report...
✅ Report saved: /content/drive/MyDrive/final/working/Brain_Tumor_Reports/Malik_Brain_MRI_Report.pdf
